# GETTING AND LOADING A CORPUS

https://nlpforhackers.io/corpora/
http://lucumr.pocoo.org/2015/11/18/pythons-hidden-re-gems/

The goal of this step is to develop an initial list of each character and their spoken lines. Dictionaries are Hash value arbitrary, so may not be ordered the same.

In [23]:
# Select and Read a file into "f" using a list and stripping out all Project Gutenberg headers and footers

from pathlib import Path
import re
idx = 1
data_folder = Path("data/")
file_to_open = data_folder / "big.txt"
f = open(file_to_open, 'r')
first_document = list(f)
#print(first_document)

# Determine whether a Project Gutenberg Text
first_header_index = 0
second_header_index = 0
footer_index = 0
if any("GUTENBERG" in s for s in first_document):
    for first_header_index in range( len(first_document) ):
        if ( ( first_document[first_header_index].find('*END*THE SMALL PRINT!') ) != -1 ) :
            break
        else:
            for first_header_index in range( len(first_document) ):
                if ( ( first_document[first_header_index].find('START OF THIS PROJECT GUTENBERG') ) != -1 ) :
                    break        
        
    second_document = list(first_document[first_header_index + 1 :])

    for second_header_index in range( len(second_document) ):
        if ( ( second_document[second_header_index].find('www.gutenberg.org') ) != -1 ) :
            break            
    for footer_index in range( len(first_document) ):
        if ( ( first_document[footer_index].find('End of Project') ) != -1 ) :
            break
        else:
            for footer_index in range( len(first_document) ):
                if ( ( first_document[footer_index].find('End of the Project') ) != -1 ) :
                    break    
        
    print(first_header_index)            
    print(second_header_index)
    print(footer_index)     
    
    script = list()
    if (second_header_index < (first_header_index + 100)):
        script = list(first_document[first_header_index +1 + second_header_index +1 : footer_index-1])
    else:
        script = list(first_document[first_header_index +1 : footer_index-1])
else:
    script = first_document
print(script)

5521
24041
29556
['\n', '\n', '\n', '\n', 'Produced by Curtis Weyant, M and the Online Distributed\n', 'Proofreading Team at http://www.pgdp.net\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', 'HISTORY\n', '\n', 'OF THE\n', '\n', 'UNITED STATES\n', '\n', '\n', 'BY\n', '\n', '\n', 'CHARLES A. BEARD\n', '\n', 'AND\n', '\n', 'MARY R. BEARD\n', '\n', '\n', '\n', 'New York\n', '\n', 'THE MACMILLAN COMPANY\n', '\n', '1921\n', '\n', '_All rights reserved_\n', '\n', 'COPYRIGHT, 1921,\n', '\n', 'BY THE MACMILLAN COMPANY.\n', '\n', '\n', 'Set up and electrotyped. Published March, 1921.\n', '\n', '\n', '\n', '\n', 'Norwood Press\n', '\n', 'J.S. Cushing Co.--Berwick & Smith Co.\n', '\n', 'NORWOOD, MASS., U.S.A.\n', '\n', '\n', '\n', '\n', 'PREFACE\n', '\n', '\n', 'As things now stand, the course of instruction in American history in\n', 'our public schools embraces three distinct treatments of the subject.\n', 'Three separate books are used. First, there is the primary book, which\n', 'is usually a 

In [24]:
#Compile a list of speakers
r = re.compile("[A-Z0-9][A-Z0-9]+")
speakers = []
for line in script:
    mtch = r.match(line)
    if mtch:
        speakers.append(mtch.group())
#print(speakers)

#Omit speakers from the list of text
s = re.compile(r"\b[A-Z{3}\.]+\b")
spoken = list(filter(lambda i: not s.search(i), script))

print(spoken)

['\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', 'New York\n', '\n', '\n', '1921\n', '\n', '_All rights reserved_\n', '\n', '\n', '\n', '\n', 'Set up and electrotyped. Published March, 1921.\n', '\n', '\n', '\n', '\n', 'Norwood Press\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', 'As things now stand, the course of instruction in American history in\n', 'our public schools embraces three distinct treatments of the subject.\n', 'Three separate books are used. First, there is the primary book, which\n', 'is usually a very condensed narrative with emphasis on biographies and\n', 'anecdotes. Second, there is the advanced text for the seventh or eighth\n', 'grade, generally speaking, an expansion of the elementary book by the\n', 'addition of forty or fifty thousand words. Finally, there is the high\n', 'school manual. This, too, ordinarily follows the beaten path, giving\n', 'fuller accounts of the sam

In [25]:
#Concatenate lines into list entries for future sentence splitting

newLines = []
singleLine = ''
singleLines = []

#Remove all line returns(ok)
for j in range(0, len(spoken)):
    spoken[j] = spoken[j].replace('\n', '')
    
#Split 5 lines at a time into new list
for k in range( 0, len(spoken), 3):
    newLines = []
    for line in range( 0, 3 ):
        try:
            newLines.append(' '+spoken[line+k])
        except:
            print("Index Error at", k, line)
            break
    #Join 5-line groups into one line and append to a list
    singleLine = ''.join(newLines)
    singleLines.append(singleLine)

#print(singleLines)

In [26]:
#Create list of sentences
sentences = []
for m in range(0, len(singleLines)):
    mtch = re.findall("[A-Z][^\.!?]*[\.!?]", singleLines[m], re.M|re.I)
    if mtch:
        sentences.append(mtch)
#print(sentences)

In [27]:
#Clean the stopword list
stoplist = []
clean_line = []
data_folder = Path("data/")
file_to_open = data_folder / "snowball_stop.txt"
f = open(file_to_open, 'r')
full_stop = list(f)

for n in range( 0, len(full_stop), 1 ):
    clean_line = full_stop[n].split('|')
    stoplist.append(clean_line[0])

for p in range(len(stoplist)):
    stoplist[p] = stoplist[p].replace('\n', '')
    
#print(stoplist)

In [28]:
#Create list of phrases using stopwords
phrases = []
candidate_phrases = []

for q in range(len(sentences)):
    for r in sentences[q]:
        words = re.split("\\s+", r)
        previous_stop = False
 
        # examine each word to determine if it is a phrase boundary marker or part of a phrase or alone
        for w in words:
 
            if w in stoplist and not previous_stop:
                # phrase boundary encountered, so put a hard indicator
                candidate_phrases.append(";")
                previous_stop = True
            elif w not in stoplist and len(w) > 3:
                # keep adding words to list until a phrase boundary is detected
                candidate_phrases.append(w.strip())
                previous_stop = False
 
    # get a list of candidate phrases without boundary demarcation
    phrases = re.split(";+", ' '.join(candidate_phrases))

# Clean up phrases    
re2 = re.compile('[^\.!?,"(){}\*:]*[\.!?,"(){}\*:]')
for s in range(len(phrases)):
    phrases[s] = re.sub(re2, '', phrases[s])
    phrases[s] = phrases[s].strip(' ')
    phrases[s] = phrases[s].replace(' ', '_')
    phrases[s] = phrases[s].replace('__', '_')
    phrases[s] = phrases[s].strip('_')

for s in range(len(phrases)):
    try:
        phrases.remove('')
        phrases.remove(' ')
        phrases.remove('/n')
    except:
        pass
    
for t in range(len(phrases)):
    print(phrases[t])

course
instruction
American_history
public_schools_embraces_three_distinct_treatments
usually
condensed_narrative
emphasis
biographies
expansion
elementary_book
addition
forty
fuller_accounts
events
assume
children_obtain_permanent_possessions
their_study
history
high_school_texts
algebra
geometry
include
multiplication_table
There
ready_answer
teachers_have_learned
bitter_experience
little_history_their_pupils_retain
they_pass_along
Still
standing_challenge
existing_methods
historians_assume
grave_responsibility
adding_their_subject
words--then_history_deserves
sharp_criticism
receiving
teachers
condition
affairs_find_justification
offering
high_school_text
time-honored_stories
exploration
biographies
Smith
time_they_reach
useless
tell
stories
perhaps
worse
demonstrated
progressive
next_place_have_omitted
descriptions
reasons
strategy
campaign
single_battle
usually
matter
operations
writers
teachers
dispose
Gettysburg
Wilderness
lines
pages_equally_absurd
serious_student
compares
ordi

America
pretensions
circle_grated
privileges_granted
often
expense
much
deepen_popular_antipathy
adherents
reappearance
formidable_union
church
stirred_anew
ancient_wrath
steady_growth
practice
introduced
Virginia
corporation
meet
given
royal_charter
Thus
process
another_every
colonies_secured
true
provision
suffrage_finally_restricted
property_owners
leaning_toward
freeholder_owning
least_fifty_acres
house
twenty-five_acres
1691
freeholder
estate_worth_forty_shillings
year
least
property
value
persons_worth
least_fifty_pounds
Restrictions_like
undoubtedly_excluded
suffrage
considerable_number
particularly
mechanics
artisans
were
means_content
relatively_easy
acquire
cheap
abundant_land
fact
large_proportion
acquired
battles
royal
interest
grants
money
official_salaries
insisted_upon_electing
treasurer
mysteries
well
management
legislature_also_used_power
money_grants
force
governor
sign_bills
Contests
Legislatures
Franklin_relates
amusing_story
Pennsylvania_assembly_held
hand
bill
exe

convinced
offensive_acts
civil
military_powers
the_traitors
threatened
condign_punishment
abettors
save
Later
Parliament_passed
sweeping_destroying
trade
intercourse
Congress_silent
Drifting
Although
Congress
given
hope
reconciliation
spring
summer
firmly_resolved
defend_American_rights
arms
militiamen
battle
Continental_army
selected_Washington
carry
diplomatic_relations
Events_followed_thick
stubborn_defense
showed
make_British
army
British_government_concluded
treaty
Landgrave
Hesse-Cassel
thousands
soldiers
many_pieces
Such
view
friends
colonies
sides
subdue
essentially_English_population_beyond
Atlantic_made_reconciliation_hopeless
coast
thrilling_story
compelling_Lord_Howe
sail
entire_army
Growth
Public_Sentiment
Favor
constitution_toward
were_drawn
path
parts
country
among
question
was
sudden_passion
work
bold
eloquent_pamphleteer_broke
upon
hesitating_public
program
without_fears
first
passionate_attack_upon
British_monarchy
equally_passionate_plea
many
condemned_monarchy_itsel

electors
numerous_branch
Thus_they_accepted
suffrage_provisions
Balance
Planting
real_division
convention
large
small_states
planting_section_founded
slave_labor
planting_states_neither
free_white_population
wealth
them
Dependent
their_prosperity_mainly_upon
sale
they_feared
afraid
majority_might
unfair_burden
possible_representation
time
restrain
taxing_power
devices_were_thought_adapted
number
obvious_reasons
Northern_delegates_objected
Once
compromise_proved
three-fifths
them
counted
purposes--representation
project
confer_upon_Congress
power
regulate_interstate
manufacturing
trading_states
enable_Congress
protect_American_manufactures
foreign_discriminations
tariffs_might_interfere
free_exchange
produce
plantations
navigation_acts_might_confine
carrying_trade
feared_might_heavily_taxed
result
related_controversies
debate
merits
heaven
states
Mason
Slavery_discourages_arts
They_prevent
migration
whites
really_strengthen
enrich
Ellsworth
Connecticut_took
ground
convention
meddle
mora

representative
Secretary
aided
Federalists
establishing
power
executive
legislative_branches
chosen
Jefferson
long
author
Declaration
Independence_retired
stern_Chief_Justice_continued
announce
Federalist_principles
name
entire_Court
doctrine
judges_have
power
declare
Congress_null
void
their_opinion_violates
Though_many_able_held
judicial_branch
government_enjoyed
principle
positively_established_rendering
opinion
Marshall_cited
sought
foundations
argument
rested
general_nature
name
United_States
limits
powers
Congress
defines
rights
Constitution_disappears
measures
violate
From
practice
federal
state_courts
passing_upon
constitutionality
This_doctrine_received
Jefferson
many
followers
Constitution
them_alone
right
prescribe_rules
government
unelected
independent
mere_thing
hands
judiciary
they_twist
shape
independent
king
executive_alone
good_thing
independence
will
nation
least
though
time
time
clinging
likewise_opposed
exercise
Courts
high_power
passing_upon
constitutionality
acts


...

Very_soon
close
Civil
them_began
protest_vigorously
policies
conduct
They_accused_Grant
the_powers
opportunities
high_office
promotion
places
power
government
machinery
corruption
considerable_number
Republicans_accepted
views
though_indorsed
utterly_routed
died
regarded
they_insisted
season
reformers
desired
party
abuses
remain_loyal
their_work
Mugwumps
Cleveland_Democracy
secession
they_were_stripped
support
supremacy_established
last_federal_soldier_withdrawn
Southern_capitals_they_succeed
capturing
opportune_moment
them_came
1884
number
vigorous
popular_leader
fire
reformers
sphere
governor
York
widely_celebrated
fine_ideals
laughed
them
carpet_knights
they
following
campaign
took_place
year
savage
Issues_were_thrust
favorite_resource
Democrats_insisted
the_Republican_party
principle_concerned
they
hardly_find_words
Republicans_retaliated
denounced
fraud
violence_practiced
Democracy
Perhaps_never
history
political_campaigns
discussions
platform
press_sink
their
they_time
reflect
ev

cause
beginning
the_overthrow
security
repose
Crown
champions
free_silver_replied
They_accused
gold_advocates
being
aggressors
assailed
labor
homes
voiced_their_sentiments
declared
was
holy
cause
liberty--the_cause
holders
idle_capital
Then_named
much
business
attorney
country_town
much
business
corporation_counsel
merchant
farmer
much
business
goes_upon
board
trade
bets_upon
price
thousand_feet_upon
cliffs
much_business
financial_magnates
back_room_corner
money
come
fighting
defense
have_petitioned
have_begged
they_have_mocked
longer
entreat
more
petition
'You_shall
press_upon
brow
labor
crown
shall
crucify_mankind_upon
cross
espoused
Southern_cause
cheering_delegation
renew
candidate
South
West_triumphed
standard
Democrats_gave_heed
platform
unlike
explicit
language
national_banks
issue_notes_intended
circulate
money
ground
derogation
recalling_Jackson's_famous_attack
Bank
duties
for
purpose
demanding
free_coinage
recurred
practice_abandoned
income_came_next
might_reversed
appeal
lab

build
colossal_fortunes
--challenges
predecessors
consider_worthy
notice--President_Roosevelt_refused
pass_without
truth
common_saying
rich_were_growing_richer
ever
history
conferred_upon
Nevertheless_felt
grave_dangers
safety
happiness
people_lurked
great_inequalities
prevent
heaping
generally_viewed
Populist
When_President_Roosevelt_turned
field
opinion_found_himself
were
advanced
members
party
results_depended_upon
making
bore
stamp
though
hardly_said
dominated_Congress
degree
damages
injuries_sustained
engaged
trainmen
Reclamation
less
railway
emphasized
issue
been_discussed
suggestion_evoked
immediate_response
Reclamation
providing
redemption
desert_areas
sluiceways
hold_water
divert
needed
Furthermore_stipulated
rents_paid
water_users
reclamation_fund
continue
Construction_started_immediately
terms
Within_seventeen_years
000_acres_been_reclaimed
irrigated_districts_were_valued
control
national_forests
trained
Bureau
Forestry--a_recommendation_carried
1907
administration
science
f

agents
employers_attended
conferences
talk
matters
recreation
members
shop_committee_decided
favor
they_attempted
settle
matter
grand_meeting
employees
broken
calling
impartial_arbitrators_selected
sides
among_citizens_outside
given
voice
decisions_affecting_their_work
welfare
rights
grievances_were_treated
matters
mutual_interest_rather
were_rigidly_excluded
negotiations
employers
Another_proposal
drawing_capital
labor_together
supplement
wage_system
definite_period
Again_they_were_given
certain_percentage
employees_were_allowed
stock
easy_terms
thus_become_part_owners
extensive_profit-sharing_become
1914
appointed
deemed_worthy
undoubtedly_growing
Labor_Managers
dealing
grievance
hundreds
companies_recognized
universities_were_giving_courses
instruction
subject
young
year
national_conference
employment_managers_held
welfare_work
every_kind_designed
make
conditions
mills
factories_safer
idea
they_were_dealing_merely
individual_employees
their_obligations_ended
payment
wages_they
take


will_faithfully_execute
office
President
will
best
defend
Constitution
shall_have_power
grant_reprieves
pardons
offences
except
cases
courts
heads
expire
care
shall_commission
officers
United_States_shall_removed
office
high_crimes
inferior_courts
Congress
time
time_ordain
receive
their_services
shall
diminished
their_continuance
public_ministers
consuls
State_shall
exceptions
regulations
trial_shall
place
places
Congress
levying
adhering
giving_them
testimony
witnesses
confession
Congress_shall_have_power
declare
punishment
attainder
treason_shall_work_corruption
blood
forfeiture_except
life
judicial_proceedings
every
Congress
general_laws_prescribe
manner
privileges
immunities
citizens
delivered
removed
State_having_jurisdiction
legislatures
States_concerned
well
shall_deprived
equal_suffrage
adoption
shall
valid
United_States
land
judges
anything
Constitution
laws
State
oath
affirmation
support
Constitution
religious_test_shall_ever_required
qualification
office
public_trust
ratific

In [29]:
# Phrase frequency count
from operator import itemgetter
wordfreq = []
for u in range(len(phrases)):
    utterance = phrases[u]
    uttcnt = 0
    uttcnt = phrases.count(utterance)
    if uttcnt > 1:
        wordfreq.append(uttcnt)
    
zipped = list(zip(phrases, wordfreq))
sortzip = sorted(zipped, key=itemgetter(1), reverse=True)

for v in range(len(sortzip)):
    print(sortzip[v])

('turning', 80)
('Europe', 80)
('William_Penn', 80)
('existed_across', 80)
('land_tenure_found', 80)
('they_sprang', 80)
('center', 80)
('record_like', 80)
('read_religious_works', 80)
('public_schools', 80)
('public_events', 80)
("read_Thomas_Paine's_thrilling_call", 80)
('much', 80)
('solid', 80)
('attack', 80)
('well_understood', 80)
('difficult', 80)
('Samoset_teaching', 80)
('Great_Britain', 80)
('World', 80)
('adventure_appealed', 80)
('grew_accustomed', 80)
('Character', 80)
('year', 80)
('royal_officers', 80)
('Lords', 80)
('Protests', 80)
('outburst', 80)
('Commons', 80)
('authority_were_null', 80)
('affairs', 80)
('crown', 80)
('withdrawal', 80)
('creating', 80)
('cargo', 80)
('majesty', 80)
('are', 80)
('every_knew', 80)
('proceeds_applied', 80)
('patriots', 80)
('last', 80)
('breast', 80)
('home', 80)
('crops_without_enjoying', 80)
('knows', 80)
('balance', 80)
("Washington's_troops_were_encouraged", 80)
('seven_members', 80)
('radical_departure', 80)
('continent', 80)
('ca

('bring', 20)
('lose_primacy', 20)
('Describe', 20)
('Attacks', 20)
('Massachusetts_took_place', 20)
('power', 20)
('purpose', 20)
('Adams', 20)
('Other_Presidents_discharged', 20)
('especially', 20)
('country_plunged', 20)
('financiers', 20)
('terrible_force', 20)
('well', 20)
('Give', 20)
('nationalism_binding_communities', 20)
('pleased', 20)
('territory', 20)
('our_title', 20)
('territory_admitted', 20)
('western_scouts_searched', 20)
('naturally_resulted', 20)
('seaboard_colonists', 20)
('triumph', 20)
('industry', 20)
('public', 20)
('private_efforts', 20)
('support', 20)
('papers_recorded', 20)
('1828', 20)
('sale', 20)
('wanted', 20)
('told_been_paid', 20)
('indeed', 20)
('influence', 20)
('Decline', 20)
('North', 20)
('kind', 20)
('came', 20)
('fifty_thousand', 20)
('petition', 20)
('turning', 20)
('itself_existed', 20)
('Election', 20)
('promise', 20)
('officers', 20)
('invasions', 20)
('themes', 20)
('1852', 20)
('Staid_Boston_hanged', 20)
('resolved', 20)
('ground', 20)
('d

('historians', 12)
('precipitating', 12)
('Articles', 12)
('period', 12)
('huge_estate', 12)
('strange_mixture', 12)
('grew_discontented', 12)
('spirit', 12)
('thought', 12)
('traveler', 12)
('urging_upon_Eastern_friends', 12)
('discovery', 12)
('extend', 12)
('Erie_Canal', 12)
('early_history', 12)
('just', 12)
('intrusted', 12)
('vote', 12)
('Trinity', 12)
('discrimination', 12)
('property_qualifications_were_thoughtlessly_imposed', 12)
('size', 12)
('virtuous', 12)
('strongest_evidence', 12)
('invested', 12)
('attacked', 12)
('remained_obdurate', 12)
('government', 12)
('Senators', 12)
('culture', 12)
('Adams', 12)
('sleeping', 12)
('feeling', 12)
('mechanics', 12)
('mills', 12)
('enforcing', 12)
('powerful_orations', 12)
('charter', 12)
('unconstitutional', 12)
('Daniel_Webster', 12)
('Jefferson', 12)
('Harrison', 12)
('clergy', 12)
('equal_duty', 12)
('United_States_favorably', 12)
('expense', 12)
('place', 12)
('1836_three_prosperous_centers', 12)
('frontier', 12)
('states', 12)


('Americans_favored_independence', 7)
('anguish', 7)
('refugees', 7)
('their_leaders_they_characterized', 7)
('rank', 7)
('fervently', 7)
('gloom_were', 7)
('times', 7)
('retreat', 7)
('capture', 7)
('seldom', 7)
('kept_British_merchantmen', 7)
('comfortable_bases', 7)
('interested', 7)
('fire', 7)
('Washington', 7)
('arms_been', 7)
('American', 7)
('records', 7)
('local_bands', 7)
('wounds', 7)
('Their_troops', 7)
('colonies', 7)
('time_were', 7)
('America_indifferent', 7)
('cause', 7)
('rendered_service', 7)
('material', 7)
('poured', 7)
('staggered', 7)
('foreign_powers_drawn', 7)
('powerful_French_fleet_shut', 7)
('brightens', 7)
('futile_struggle', 7)
('hold_upon', 7)
('Americans', 7)
('pamphlets', 7)
('verge', 7)
('This_federal_republic_born', 7)
('well', 7)
('statesmen', 7)
('debt', 7)
('undertake', 7)
('American_leaders', 7)
('Thomas_Jefferson--emphasizing', 7)
('industry', 7)
('Congress_composed', 7)
('legislature_might_direct', 7)
('forceful', 7)
('transaction', 7)
('payment'

('equal_division', 5)
('attempts', 5)
('victorious', 5)
('standing_together', 5)
('they_also_recognized', 5)
('their_cargoes', 5)
('products', 5)
('railways', 5)
('Philadelphia', 5)
('prices', 5)
('made_profits', 5)
('less', 5)
('half', 5)
('manufacturing_states_have_reduced', 5)
('rest', 5)
('foreign_manufactures', 5)
('stand_diametrically', 5)
('every_diminution', 5)
('ever_knew', 5)
('menaced', 5)
('theory_sound', 5)
('complete_Democratic_triumph', 5)
('What_political', 5)
('economic_conditions', 5)
('Slavery', 5)
('were', 5)
('slaves', 5)
('With_singleness', 5)
('hissed', 5)
('borders--no_pirate', 5)
('strand', 5)
('motion_rescinding', 5)
('receded', 5)
('Young_statesmen', 5)
('take', 5)
('conviction', 5)
("Washington's", 5)
('capital', 5)
('influence', 5)
('overwhelming_proportion', 5)
('five_were_called', 5)
('them_found_wanting', 5)
('challenged', 5)
('whether_slavery', 5)
('expressly_guaranteed', 5)
('something', 5)
('benefit', 5)
('heavy_borrowers', 5)
('immense_gain', 5)
('Co

('began_operations', 3)
('town', 3)
('acquired_enormous_holdings', 3)
('valuable_spot_within', 3)
('high_enough', 3)
('prepared', 3)
('important_source', 3)
('scarcity', 3)
('made_impossible', 3)
('European_system', 3)
('stony_soil', 3)
('poured', 3)
('settling', 3)
('redemptioner', 3)
('discharged_bond', 3)
('published_entertaining_Recollections', 3)
('features', 3)
('western_Tennessee_they', 3)
('descriptions', 3)
('methods', 3)
('nights', 3)
('stopping_every_evening', 3)
('brink', 3)
('When', 3)
('suffrage', 3)
('surrendered', 3)
('government', 3)
('leaders', 3)
('Congress', 3)
('Congress', 3)
('constitution', 3)
('constitutions', 3)
('their_armies', 3)
('Union_virtually_dissolved_right', 3)
('separation', 3)
('kindlier_reception', 3)
('fields', 3)
('small_farmers', 3)
('ringing', 3)
('Over', 3)
('There', 3)
('undoubted_source', 3)
('accomplish', 3)
('enterprises', 3)
('active', 3)
('bones', 3)
('much', 3)
('brought_across', 3)
('rivers', 3)
('Atlantic', 3)
('beyond', 3)
('five-ton_

('immediate_separation', 2)
('many_wars', 2)
('them', 2)
('weighty', 2)
('managed', 2)
('power', 2)
('distant', 2)
("nature_cries_''tis_time", 2)
('contest', 2)
('choice', 2)
('never_shone', 2)
('virtuous_supporter', 2)
('exclaimed', 2)
('Sound_doctrine', 2)
('idea', 2)
('independence_began', 2)
('responded', 2)
('British_flag', 2)
('Continental_Congress_alive', 2)
('Virginia', 2)
('Others_hesitated', 2)
('couriers_swiftly_carried', 2)
('thrones', 2)
('armies', 2)
('apart', 2)
('throw', 2)
('That_section', 2)
('giving', 2)
('every_form', 2)
('This', 2)
('British_empire', 2)
('With', 2)
('adopt', 2)
('form', 2)
('Congress', 2)
('complete_constitution', 2)
('legislature', 2)
('except', 2)
('senate', 2)
('revolutionary_leaders_were', 2)
('John_Adams', 2)
('Jersey_women_were_allowed', 2)
('serve', 2)
('guide', 2)
('begin', 2)
('state_constitutions', 2)
('establishment', 2)
('sagacious_leaders_began', 2)
('urge', 2)
('desirability', 2)
('committee', 2)
('draft', 2)
('states', 2)
('Articles'

# TF-IDF

https://cdn-images-1.medium.com/max/800/1*nSqHXwOIJ2fa_EFLTh5KYw.png

In [ ]:
# Computes ratio of word's appearances to total words
def computeTF(wordDict, bow): 
    bowCount = len(bow) #BOW = Bag of Words
    tfDict = {}
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

In [ ]:
# Calculates the weight of rare words across all docs
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
                
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
    
    return idfDict

In [ ]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf